In [15]:
!apt-get update
!apt-get install -y libsdl2-dev
!apt-get install -y xvfb

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists

In [9]:
%%writefile HenonMap.cpp

#include <iostream>
#include <SDL2/SDL.h>

const int SCREEN_WIDTH = 640;
const int SCREEN_HEIGHT = 480;

const int MAX_ITERATIONS = 1000000;
const float ALPHA = 1.4;

int main(int argc, char* argv[]) {

    // Initialize SDL
    SDL_Init(SDL_INIT_VIDEO);

    // Create a window
    SDL_Window* window = SDL_CreateWindow("Henon Map", SDL_WINDOWPOS_UNDEFINED, SDL_WINDOWPOS_UNDEFINED, SCREEN_WIDTH, SCREEN_HEIGHT, SDL_WINDOW_SHOWN);

    // Create a renderer
    SDL_Renderer* renderer = SDL_CreateRenderer(window, -1, SDL_RENDERER_ACCELERATED);

    // Wait for user input
    SDL_Event e;
    bool quit = false;
    while (!quit) {
        while (SDL_PollEvent(&e) != 0) {
            if (e.type == SDL_QUIT) {
                quit = true;
            }
            else if (e.type == SDL_MOUSEBUTTONDOWN) {
                int x = e.button.x;
                int y = e.button.y;

                // Calculate and plot Henon map trajectory
                float x_old = (float)x / SCREEN_WIDTH * 4 - 2; // Map x to [-2, 2]
                float y_old = (float)y / SCREEN_HEIGHT * 4 - 2; // Map y to [-2, 2]
                for (int i = 0; i < MAX_ITERATIONS; i++) {
                    float x_new = x_old * cos(ALPHA) - y_old * sin(ALPHA) + x_old * x_old * sin(ALPHA);
                    float y_new = x_old * sin(ALPHA) + y_old * cos(ALPHA) - x_old * x_old * cos(ALPHA);
                    x_old = x_new;
                    y_old = y_new;

                    int pixel_x = (x_new + 2) / 4 * SCREEN_WIDTH; // Map x back to [0, SCREEN_WIDTH]
                    int pixel_y = (y_new + 2) / 4 * SCREEN_HEIGHT; // Map y back to [0, SCREEN_HEIGHT]

                    SDL_SetRenderDrawColor(renderer, 255, 255, 255, 255); // Set color to white
                    SDL_RenderDrawPoint(renderer, pixel_x, pixel_y); // Plot pixel
                }
                SDL_RenderPresent(renderer); // Update screen
            }
        }
    }

    // Clean up and quit
    SDL_DestroyRenderer(renderer);
    SDL_DestroyWindow(window);
    SDL_Quit();
    return 0;
}

Overwriting HenonMap.cpp


In [ ]:
! g++ -o HenonMap HenonMap.cpp -lSDL2
! xvfb-run -s "-screen 0 640x480x24" ./HenonMap